In [64]:
json_path = "data/annotations/caption/train/20230707_8_SN46_T1/overhead_view/20230707_8_SN46_T1_caption.json"
video_root = "data/videos"

In [2]:
import cv2
from PIL import Image
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration,AutoModelForCausalLM 
import os
import json


/home/rornelas/Desktop/Mi3_Lab/AI_CIY_CHALLENGE/llava-vlm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Videollama3
device = "cuda:0"
model_path = "DAMO-NLP-SG/VideoLLaMA3-2B"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    device_map={"": device},
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-2B:
- image_processing_videollama3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-2B:
- processing_videollama3.py
- image_processing_videollama3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: cannot import name 'VideoInput' from 'transformers.image_utils' (/home/rornelas/Desktop/Mi3_Lab/AI_CIY_CHALLENGE/llava-vlm/lib/python3.12/site-packages/transformers/image_utils.py)

In [4]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

True
12.1
1
NVIDIA GeForce RTX 4080 SUPER


In [3]:
# Llava
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id, use_fast=True)
tokenizer = AutoProcessor.from_pretrained(model_id, use_fast=True)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True
).to("cuda" if torch.cuda.is_available() else "cpu")

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 40.24it/s]


In [93]:
def generate_caption(frames, prompt):
    captions = []
    for frame in frames:
        inputs = processor(images=frame, text=prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=150)
        caption = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        captions.append(caption)
        
        return " ".join(captions)

In [5]:
def extract_middle_frame(video_path, start_time, end_time):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[ERROR] Cannot open video: {video_path}")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        cap.release()
        return None
    
    middle_time = (start_time + end_time) / 2.0
    frame_number = int(middle_time * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print(f"[ERROR] Failed to read frame at {frame_number} in video: {video_path}")
        return None

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    return pil_image

In [66]:
def extract_frames(video_paths, start_time = 0.0, end_time=None, interval=1.0):
    all_frames = []
    for video_path in video_paths:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            print(f"Cannot open video: {video_path}")
            continue
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0:
            cap.release()
            continue
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        video_duration = total_frames/fps
        
        if end_time is None or end_time > video_duration:
            end_time = video_duration
        
        frame_interval = int(fps*interval)
        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)

        for frame_num in range(start_frame, end_frame, frame_interval):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            success, frame = cap.read()
            if not success:
                continue
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame_rgb)
            all_frames.append(pil_image)
        cap.release()
    return all_frames

In [6]:
def find_video_path(video_root, video_file):
    for subfolder in ["train", "val"]:
        folder_path = os.path.join(video_root, subfolder)
        for root, _, files in os.walk(folder_path):
            if video_file in files:
                return os.path.join(root, video_file)
    return None

In [7]:

def find_video(video_root: str, video_file: str):
    for dirpath, _, files in os.walk(video_root):
        if video_file in files:
            return os.path.join(dirpath, video_file)
    return None

In [114]:
def run_on_json(json_path: str, video_root: str):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    video_files = data.get("overhead_videos", [])
    events = data.get("event_phase", [])

    if not video_files or not events:
        print(f"[SKIP] Invalid video/event alignment in {json_path}")
        return

    print(f"[INFO] Processing: {os.path.basename(json_path)}")

    for event in events:
        start = float(event["start_time"])
        end = float(event["end_time"])
        labels = event.get("labels", [])

        all_frames = []
        for video_file in video_files:
            video_paths = [find_video(video_root, vf) for vf in video_files]
            frames = extract_frames(video_paths, start, end)
            if frames:
                all_frames.extend(frames)

        if not all_frames:
            continue

        caption_ped = generate_caption(
            all_frames,
            "<image>  Describe the crash victim in detail: age, gender, clothing, posture, and behavior. Were they distracted or alert? Were they in a legal crossing area? Mention their awareness of the vehicle."

        )
        caption_veh = generate_caption(
            all_frames,
            "<image>  Describe the vehicle involved in the accident with the crash victim. Was it accelerating, braking, or turning? Describe its position relative to the pedestrian and crosswalk. Did it yield? Was it following traffic rules?"
        )
        print(f"\nEvent Labels: {labels}")
        print(f"Pedestrian: {caption_ped}")
        print(f"Vehicle: {caption_veh}")

In [115]:
#Llava
run_on_json(json_path, video_root)

[INFO] Processing: 20230707_8_SN46_T1_caption.json

Event Labels: ['4']
Pedestrian: Describe the crash victim in detail: age, gender, clothing, posture, and behavior. Were they distracted or alert? Were they in a legal crossing area? Mention their awareness of the vehicle.

The crash victim is a person wearing a white shirt. They are standing in the middle of the street, possibly waiting to cross the road. They are not in a legal crossing area, and their posture suggests that they are alert and aware of their surroundings. It is not clear if they were distracted or not, but their presence in the middle of the street indicates that they were not following traffic rules and could be at risk of being hit by a vehicle.
Vehicle: Describe the vehicle involved in the accident with the crash victim. Was it accelerating, braking, or turning? Describe its position relative to the pedestrian and crosswalk. Did it yield? Was it following traffic rules?

The vehicle involved in the accident was a b

In [39]:
#Beginning the vqa
json_path = "data/annotations/vqa/train/20230707_8_SN46_T1"
video_root = "data/videos"

In [11]:
def find_vqa_type(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    if isinstance(data, list) and all('environment' in item for item in data if isinstance(item, dict)):
        return "environment"
    
    if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
        item = data[0]
        if "event_phase" in item:
            if "overhead_videos" in item:
                return "overhead_view"
            elif "vehicle_view" in item:
                return "vehicle_view"

In [124]:
json_path2 = 'data/annotations/vqa/train/20230707_8_SN46_T1/overhead_view/20230707_8_SN46_T1.json'
test = find_vqa_type(json_path2)
print("Detected the question type: ", test)

Detected the question type:  overhead_view


In [21]:
def generate_answer(model, image_pil, question, choices, tokenizer, processor):
    prompt = "<image>\n" + question + "\nChoices:n"
    for key, val in choices.items():
        prompt += f"{key}: {val}\n"
    prompt += "Answer with the letter of the correct choice."
    
    inputs = processor(text=prompt, images=image_pil, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10)
        answer_text = tokenizer.decode(outputs[0], skip_special_tokens=True).lower().strip()
    
    for letter in ['a', 'b', 'c', 'd']:
        if letter in answer_text:
            return letter
    return None    

In [131]:
def final_answer(model, frames, question, choices, processor, tokenizer):
    votes = {}
    
    for frame in frames:
        answer = generate_answer(model,frame, question, choices, tokenizer, processor)
        if answer:
            votes[answer] = votes.get(answer, 0) + 1
            
    if not votes:
        return None

    return max(votes.items(), key=lambda x: x[1])[0]

In [142]:
def run_vqa_on_scenario(base_dir, video_root, model):
    all_results = []
    
    for root, dirs, files in os.walk(base_dir):
        if any(p in root for p in ['environment', 'overhead_view', 'vehicle_view']):
            for file in files:
                if file.endswith(".json"):
                    perspective = os.path.basename(root)
                    scenario_id = file.replace(".json", "")
                    full_path = os.path.join(root, file)
                    
                    print(f"Processing {perspective}")
                    
                    with open(full_path, 'r') as f:
                        json_data = json.load(f)
                    
                    if perspective == "environment":
                        result = evaluate_environment(json_data, video_root, model)
                    elif perspective == "overhead_view":
                        result = evaluate_overhead(json_data, video_root, model)
                    elif perspective == "vehicle_view":
                        result = evaluate_vehicle(json_data, video_root, model)
                    
                    all_results.append({
                        "scenario": scenario_id,
                        "perspective": perspective,
                        "results": result
                        })   
    return all_results 

In [130]:
def evaluate_vehicle(json_data, video_root, model):
    results = []
    video_filename = json_data[0].get("vehicle_view")
    video_path = find_video(video_root, video_filename)
    
    for event in json_data[0].get("event_phase", []):
        start = float(event["start_time"])
        end = float(event["end_time"])
        for q in event.get("conversations", []):
            question_text = q.get("question")
            choices = {k: q[k] for k in ['a', 'b', 'c', 'd'] if k in q}
            correct = q.get("correct")
            
            frames = extract_frames([video_path], start, end)
            answer = final_answer(model, frames, question_text, choices, processor, tokenizer)
            is_correct = (answer == correct)
            
            results.append({
                "perspective": "vehicle_view",
                "question": question_text,
                "choices": choices,
                "correct": correct,
                "model_answer": answer,
                "is_correct": is_correct,
            })
    return results
    

In [129]:
def evaluate_overhead(json_data, video_root, model):
    results = []
    overhead_videos = json_data[0].get("overhead_videos", [])
    
    video_dirname = os.path.basename(json_path).replace(".json", "")
    video_prefix = video_dirname.split("_")[0:4]
    video_folder = "_".join(video_prefix)
    video_dir = os.path.join(video_root, "train", video_folder, "overhead_view")
    videos_path = [os.path.join(video_dir, vid) for vid in overhead_videos]
    
    event_phases = json_data[0].get("event_phase", [])
    for phase in event_phases:
        start = float(phase.get("start_time", 0))
        end = float(phase.get("end_time", 0))
        for conv in phase.get("conversations", []):
            question_text = conv.get("question")
            correct = conv.get("correct")
            choices = {k: conv[k] for k in ['a', 'b', 'c', 'd'] if k in conv}
            
            frames = extract_frames(videos_path, start, end)
            answer = final_answer(model, frames, question_text, choices, processor, tokenizer)
            is_correct = (answer == correct)
            
            results.append({
                "perspective": "overhead",
                "question": question_text,
                "choices": choices,
                "correct": correct,
                "model_answer": answer,
                "is_correct": is_correct,
            })
    return results

In [128]:
def evaluate_environment(json_data, video_root, model):
    results = []
    overhead_videos = json_data[0].get("overhead_videos", [])
    
    video_dirname = os.path.basename(json_path).replace(".json", "")
    video_prefix = video_dirname.split("_")[0:4]
    video_folder = "_".join(video_prefix)
    video_dir = os.path.join(video_root, "train", video_folder, "overhead_view")
    videos_path = [os.path.join(video_dir, vid) for vid in overhead_videos]
    
    questions = json_data[0].get("environment", [])
    for q in questions:
        question_text = q.get("question")
        correct = q.get("correct")
        choices = {k: q[k] for k in ['a', 'b', 'c', 'd'] if k in q}
    
        frames = extract_frames(videos_path)
        answer = final_answer(model, frames, question_text, choices, processor, tokenizer)
        is_correct = (answer == correct)
        
        results.append({
            "perspective": "environment",
            "question": question_text,
            "choices": choices,
            "correct": correct,
            "model_answer": answer,
            "is_correct": is_correct,
        })
    return results
    

In [143]:
base_dir = "data/annotations/vqa/train/20230707_8_SN46_T1"
video_root = "data/videos"

results = run_vqa_on_scenario(base_dir, video_root, model)

Processing overhead_view
Processing vehicle_view
Processing environment


In [144]:
def save_results_to_json(results, output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(results, f, indent=4)
    print(f"[INFO] Results saved to: {output_path}")

In [145]:
output_json = "outputs/full_vqa_results_test.json"
save_results_to_json(results, output_json)

[INFO] Results saved to: outputs/full_vqa_results_test.json
